In [82]:
from itertools import chain

Coord = tuple[int, int, int]
Brick = tuple[Coord, Coord]


def parse(input_string: str) -> list[Brick]:
    output = []
    for line in input_string.splitlines():
        coords = line.split("~")
        start = eval(coords[0])
        end = eval(coords[1])
        brick_points = []
        for i in range(start[0], end[0] + 1):
            for j in range(start[1], end[1] + 1):
                for k in range(start[2], end[2] + 1):
                    brick_points.append((i, j, k))

        output.append(brick_points)

    return sorted(output, key=lambda x: min([z for _, _, z in x]))


def settle_bricks(bricks: list[Brick]):
    settled = set()
    output_bricks = []
    for brick in bricks:
        min_z = 1
        max_z = min([point[2] for point in brick])
        for i in range(max_z - 1, 0, -1):
            for point in brick:
                if (point[0], point[1], i) in settled:
                    min_z = max(min_z, i + 1)
                    break

        next_positions = [(x, y, z - max_z + min_z) for x, y, z in brick]
        settled.update(next_positions)
        output_bricks.append(next_positions)

    return output_bricks


def identify_redundant_bricks(bricks: list[Brick]) -> int:
    positions = set(chain.from_iterable(bricks))
    supports = []

    for brick in bricks:
        structural = set()
        for x, y, z in brick:
            if (x, y, z + 1) in positions and (x, y, z + 1) not in brick:
                index = next(i for i, b in enumerate(bricks) if (x, y, z + 1) in b)
                structural.add(index)

        supports.append(structural)

    redudant_bricks = []
    for id in range(len(bricks)):
        support = supports[id]
        if len(support) == 0 or all(
            list(chain.from_iterable(supports[:id] + supports[id + 1 :])).count(i) > 0
            for i in support
        ):
            redudant_bricks.append(id)

    return redudant_bricks

In [83]:
test_input = """\
1,0,1~1,2,1
0,0,2~2,0,2
0,2,3~2,2,3
0,0,4~0,2,4
2,0,5~2,2,5
0,1,6~2,1,6
1,1,8~1,1,9"""

bricks = settle_bricks(parse(test_input))
assert len(identify_redundant_bricks(bricks)) == 5

In [84]:
bricks = settle_bricks(parse(open("22.txt").read()))
value = len(identify_redundant_bricks(bricks))

print(f"Part 1: {value}")
assert value == 465

Part 1: 465
